# Configuring IP over UDP

This example notebook shows how a user can configure the IP address of their board with UDP. 

**A custom register configuration file (*register-config.yml*) will be generated by the notebook and must be used for future UDP connections to the board for as long as the IP configured with this notebook is valid or until the next reboot of the board.**


## Naludaq Version

MinVersion: 0.33.2

In [ ]:
# Print Naludaq version 
import naludaq
print(f"Naludaq version: {naludaq.__version__}")

## Compatible Boards

* This example applies to all board models with an Ethernet port.

## Imports and Variables

In [ ]:
from naluconfigs import get_available_models, get_configuration
from naludaq.board import Board, startup_board
from naludaq.communication import ControlRegisters
from naludaq.controllers import get_connection_controller
from naludaq.tools.ftdi import list_ftdi_devices

import os
import yaml

## Logging

The logger setup below will show additional information from the NaluDAQ package. This is useful for debugging purposes, but
may be skipped if not needed.

In [ ]:
import logging


def setup_logger(level=logging.INFO):
    """Setup a basic logger."""
    logger = logging.getLogger()
    handler = logging.StreamHandler()
    handler.setFormatter(
        logging.Formatter("%(asctime)s %(name)-30s [%(levelname)-6s]: %(message)s")
    )
    logger.addHandler(handler)
    logger.setLevel(level)
    suppress = [
        "naludaq.UART",
        "naludaq.FTDI",
    ]
    for name in suppress:
        logging.getLogger(name).setLevel(logging.CRITICAL)
    return logger

logger = setup_logger()

try:
    logger.debug("logger already setup")
except:
    logger = setup_logger()

## Options

The following parameters must be provided to successfully set the IP address:


In [ ]:
# Determining Board Model
print(get_available_models())

A list of all FTDI devices and their serial numbers can be found using the following command:

In [ ]:
print(list_ftdi_devices())

- **board_model**:      : *board_model*
  - Valid options can be found using the command in a previous code block.

- **workspace_folder**: *folder_name*
  - The folder where ADC data is saved. A workspace must be specified to successfully initiate a connection to the board but no data will be collected with this notebook example.

- **serial_number**: *serial_number*
  - The serial number of the FTDI device. A list of FTDI devices and their serial numbers can be found using a previous code block.

- **register_config_file**: *file_name*
  - The name of the custom register configuration file generated by this notebook. Must be a ".yaml" or ".yml" file.

- **default_board_addr**: (*ip_address*, *port*)
  - Refer to the Quick Start guide for your board.

- **target_board_addr**:  (*ip_address*, *port*)
  - This is the IP address that you would like to set for the board

- **recv_addr**:          (*ip_address*, *port*)
   - This is the IP address of the computer used to configure the IP address of the board

Set these parameters in the following code block. All file and folder names are assumed to be in the same directory as the notebook.


In [ ]:
################
# Board Config #
################

board_model = "hdsocv2_evalr2"
workspace_folder = "board_workspace"
register_config_file = "register-config.yml"
serial_number = "A503VYQF"

#############
# IP Config #
#############

target_board_addr = ("192.168.1.32", 4660)
recv_addr = ("192.168.1.198", 4660)


Loads the default register and clock configuration.

In [ ]:
board = Board(board_model)
board.load_registers()
board.load_clockfile()

Converts *target_ip_addr* into hex codes for applicable registers.

In [ ]:
src_ip, src_port = target_board_addr
src_octets = get_connection_controller(board)._get_octets_or_raise(src_ip)


eth_src_addr31_16 = (src_octets[0] << 8) | src_octets[1]

eth_src_addr15_0 = (src_octets[2] << 8) | src_octets[3]

eth_addr_sel = 3
ethernet_src_config = {
    "eth_src_addr31_16": eth_src_addr31_16,
    "eth_src_addr15_0": eth_src_addr15_0,
    "eth_addr_sel": eth_addr_sel
}

Open connection to board and write *target_ip_address* to board. 

In [ ]:
workspace_path = os.path.join(os.getcwd(), workspace_folder)

board.get_ftdi_connection(
    serial_number="A503VYQF",
    baud=115200
)

with board:
    if startup_board(board):
        ControlRegisters(board).write_many(
            ethernet_src_config
        )
        print(ethernet_src_config)
        print('Successfully configured IP Address')
    else:
        print(f"Failed to start {board_model}")

board.disconnect()

print(f"Disconnected from {board_model}")


## Creating a Register Configuration File

This custom configuration file must be loaded for future connections to the IP address configured using this notebook. Refer to connecting-to-new-ip.ipynb for usage example.

In [ ]:
register_config = get_configuration(board_model)
register_config["registers"]["control_registers"]["eth_dest_addr31_16"]["value"] = eth_src_addr31_16
register_config["registers"]["control_registers"]["eth_dest_addr15_0"]["value"] = eth_src_addr15_0
register_config["registers"]["control_registers"]["eth_addr_sel"]["value"] = eth_addr_sel

with open(register_config_file, "w") as file:
    yaml.dump(register_config, file)

print(f"Saved register configuration to {register_config_file}")